In [1]:
import sys, os

PROJECT_ROOT = r"C:\Assigment\2025-2026\Machine Learning\machine_learning_project" #thay bang file path root cua minh
SRC_PATH = os.path.join(PROJECT_ROOT, "src")

if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("SRC_PATH:", SRC_PATH)


PROJECT_ROOT: C:\Assigment\2025-2026\Machine Learning\machine_learning_project
SRC_PATH: C:\Assigment\2025-2026\Machine Learning\machine_learning_project\src


In [2]:
from pathlib import Path

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import data.before_model as before_model      
import data.data_preprocessing as data_preprocessing


In [4]:
df = pd.read_csv(r'C:\Assigment\2025-2026\Machine Learning\machine_learning_project\dataset\raw\Hanoi_Daily_Selected.csv')

In [5]:
df_hourly = pd.read_csv(r'C:\Assigment\2025-2026\Machine Learning\machine_learning_project\dataset\raw\Hanoi_Hourly_Selected.csv')
train_dataset_h, test_dataset_h = before_model.train_test_split_hourly(df_hourly)
before_model_hourly = before_model.before_model_pipeline_hourly
before_model_hourly.fit(train_dataset_h)
train_dataset_h = before_model_hourly.transform(train_dataset_h)
test_dataset_h = before_model_hourly.transform(test_dataset_h)

Dropped columns due to constant value: []
Dropped columns due to zero variance: []


c:\Users\ASUS\anaconda3\envs\wp_env\Lib\site-packages\sklearn\pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
c:\Users\ASUS\anaconda3\envs\wp_env\Lib\site-packages\sklearn\pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
c:\Users\ASUS\anaconda3\envs\wp_env\Lib\site-packages\sklearn\pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
c:\Users\ASUS\anaconda3\envs\wp_env\Lib\site-packages\skle

In [15]:
train_dataset_h = train_dataset_h.reset_index()

In [5]:
train_dataset, test_dataset = before_model.train_test_split(df)
before_model_pipeline = before_model.before_model_pipeline
before_model_pipeline.fit(train_dataset)
train_dataset = before_model_pipeline.transform(train_dataset)
test_dataset = before_model_pipeline.transform(test_dataset)

Dropped columns due to constant value: []
Dropped columns due to zero variance: []


C:\Assigment\2025-2026\Machine Learning\machine_learning_project\src\data\data_preprocessing.py:109: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data_df[col] = pd.to_numeric(data_df[col], errors='ignore')
c:\Users\ASUS\anaconda3\envs\wp_env\Lib\site-packages\sklearn\pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
C:\Assigment\2025-2026\Machine Learning\machine_learning_project\src\data\data_preprocessing.py:109: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data_df[col] = pd.to_numeric(data_df[col], errors='ignore')
C:\Assigment\2025-2026\Machine Learnin

In [ ]:
## model_training 1

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna  
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt


class MultiHorizonWalkForwardOptuna_XGBoost_Pipeline:
    def __init__(
        self,
        df,
        date_col,
        target_col,
        feature_cols,
        n_splits=5,
        test_size=30,
        mode="expanding",
        horizons=(1, 2, 3, 4, 5),   # t+1 ... t+5
    ):
        """
        df          : pandas DataFrame in chronological order
        date_col    : name of date column (datetime)
        target_col  : name of target column (e.g., 'temp')
        feature_cols: list of feature columns (SHOULD NOT include target)
        n_splits    : number of walk-forward folds
        test_size   : number of days in each test fold
        mode        : "expanding" or "rolling"
        horizons    : iterable of forecast horizons (days ahead), e.g. (1,2,3,4,5)
        """
        self.df = df.copy().sort_values(date_col).reset_index(drop=True)
        self.date_col = date_col
        self.target_col = target_col
        self.feature_cols = feature_cols
        self.n_splits = n_splits
        self.test_size = test_size
        self.mode = mode
        self.horizons = tuple(horizons)

        # For each horizon h, we will have:
        #   self.walkfolds[h]      : list of ((X_train, y_train_h), (X_val, y_val_h))
        #   self.walkfold_dates[h] : list of ((X_train_dates, y_train_dates_h),
        #                                     (X_val_dates,   y_val_dates_h))
        self.walkfolds = {h: [] for h in self.horizons}
        self.walkfold_dates = {h: [] for h in self.horizons}

        # Final model per horizon: self.final_models[h] = trained XGBRegressor
        self.final_models = {h: None for h in self.horizons}

        self.best_params = None
        self.fold_history = []  # store optuna trial results

        # Actual feature columns: remove date/target if accidentally passed in
        self.X_cols = [
            c for c in self.feature_cols
            if c not in [
                self.date_col,
                self.target_col,
                self.target_col + "_next",
                self.date_col + "_next",
            ]
        ]

    # ======================== 1. TARGET SHIFT FOR ALL HORIZONS (TRAIN) ========================

    def add_target_shifts(self):
        """
        Create shifted targets for all horizons, e.g. for target='temp':
        - temp_h1 = temp at t+1
        - temp_h2 = temp at t+2
        - ...
        Also create corresponding date columns date_h{h} for reference.
        After that, drop rows with any NaN in those shifted targets.
        """
        for h in self.horizons:
            self.df[f"{self.target_col}_h{h}"] = self.df[self.target_col].shift(-h)
            self.df[f"{self.date_col}_h{h}"] = self.df[self.date_col].shift(-h)

        target_cols = [f"{self.target_col}_h{h}" for h in self.horizons]
        self.df = self.df.dropna(subset=target_cols).reset_index(drop=True)

    # ======================== 1b. TARGET SHIFT CHO TEST SET ========================

    def prepare_test_dataset(self, test_df_raw):
        """
        Create shifted targets for all horizons on a separate test DataFrame.

        Parameters
        ----------
        test_df_raw : pd.DataFrame
            Raw test DataFrame containing at least [date_col, target_col] and feature_cols.

        Returns
        -------
        test_df : pd.DataFrame
            A new DataFrame with:
            - original columns
            - {target_col}_h{h}
            - {date_col}_h{h}
          Rows with NaN in any shifted target are dropped.
        """
        test_df = test_df_raw.copy().sort_values(self.date_col).reset_index(drop=True)

        for h in self.horizons:
            test_df[f"{self.target_col}_h{h}"] = test_df[self.target_col].shift(-h)
            test_df[f"{self.date_col}_h{h}"] = test_df[self.date_col].shift(-h)

        target_cols = [f"{self.target_col}_h{h}" for h in self.horizons]
        test_df = test_df.dropna(subset=target_cols).reset_index(drop=True)
        return test_df

    # ======================== 2. CREATE WALK-FORWARD FOLDS ========================

    def create_walkforward_folds(self):
        """
        Create walk-forward folds for each horizon.

        For each horizon h:
          self.walkfolds[h] = [
              ((X_train, y_train_h), (X_val, y_val_h)),
              ...
          ]

          self.walkfold_dates[h] = [
              ((X_train_dates, y_train_dates_h), (X_val_dates, y_val_dates_h)),
              ...
          ]
        """
        df_len = len(self.df)
        step = (df_len - self.test_size) // self.n_splits

        # reset containers
        self.walkfolds = {h: [] for h in self.horizons}
        self.walkfold_dates = {h: [] for h in self.horizons}

        for i in range(self.n_splits):
            train_end = step * (i + 1)
            test_start = train_end + 91   # 91-day gap
            test_end = test_start + self.test_size
            if test_end > df_len:
                break

            if self.mode == "expanding":
                train_start = 0
            else:  # rolling
                train_start = max(0, train_end - step * 2)

            train_df = self.df.iloc[train_start:train_end]
            test_df = self.df.iloc[test_start:test_end]

            # Features (same for all horizons)
            X_train = train_df[self.X_cols].reset_index(drop=True)
            X_val = test_df[self.X_cols].reset_index(drop=True)

            # Dates of "feature time" (t)
            X_train_dates = train_df[self.date_col].reset_index(drop=True)
            X_val_dates = test_df[self.date_col].reset_index(drop=True)

            # For each horizon, build targets & date of target
            for h in self.horizons:
                y_train_h = train_df[f"{self.target_col}_h{h}"].reset_index(drop=True)
                y_val_h = test_df[f"{self.target_col}_h{h}"].reset_index(drop=True)

                y_train_dates_h = train_df[f"{self.date_col}_h{h}"].reset_index(drop=True)
                y_val_dates_h = test_df[f"{self.date_col}_h{h}"].reset_index(drop=True)

                # Walkfolds
                self.walkfolds[h].append(
                    ((X_train, y_train_h), (X_val, y_val_h))
                )

                # Date folds
                self.walkfold_dates[h].append(
                    ((X_train_dates, y_train_dates_h),
                     (X_val_dates,   y_val_dates_h))
                )

    # ======================== 3. OPTUNA OBJECTIVE (SHARED PARAMS) ========================

    def create_objective(self):
        """
        Optuna objective.

        A single set of hyperparameters is shared for all horizons.
        The objective value is the mean of the mean-RMSE across horizons.

        - Dùng early_stopping_rounds trong constructor XGBRegressor (hợp XGBoost mới).
        - Dùng trial.report + trial.should_prune để prune trial tệ.
        """
        def objective(trial):
            params = {
                'objective': 'reg:squarederror',
                'eval_metric': 'rmse',
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'max_depth': trial.suggest_int('max_depth', 3, 6),
                'gamma': trial.suggest_float('gamma', 0.0, 1.0),
                'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 10.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
                'subsample': trial.suggest_float('subsample', 0.5, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'min_child_weight': trial.suggest_int('min_child_weight', 3, 10),
            }

            horizon_results = {}
            overall_rmse_list = []
            global_step = 0  # step index for Optuna pruning

            for h in self.horizons:
                scores = []
                fold_results = []

                for i, ((X_train, y_train), (X_val, y_val)) in enumerate(self.walkfolds[h]):

                    model = xgb.XGBRegressor(
                        **params,
                        n_estimators=800,
                        random_state=42,
                        tree_method='hist',
                        # early stopping for new XGBoost (passed via constructor)
                        early_stopping_rounds=100
                    )

                    model.fit(
                        X_train,
                        y_train,
                        eval_set=[(X_val, y_val)],
                        verbose=False
                    )

                    preds = model.predict(X_val)
                    # GIỮ LOG Y HỆT EM, NHƯNG Ở ĐÂY CHỊ DÙNG RMSE ĐÚNG NGHĨA
                    rmse = mean_squared_error(y_val, preds)
                    ss_res = np.sum((y_val - preds) ** 2)
                    ss_tot = np.sum((y_val - np.mean(y_val)) ** 2)
                    r2 = 1 - ss_res / ss_tot if ss_tot != 0 else 0.0

                    scores.append(rmse)
                    fold_results.append({'rmse': rmse, 'r2': r2})

                    print(
                        f"Trial {trial.number}, Horizon h={h}, "
                        f"Fold {i+1}: RMSE={rmse:.4f}, R²={r2:.4f}"
                    )

                    # Report to Optuna and possibly prune
                    trial.report(rmse, step=global_step)
                    if trial.should_prune():
                        print(f"Trial {trial.number} pruned at global_step={global_step}")
                        raise optuna.TrialPruned()
                    global_step += 1

                mean_rmse_h = float(np.mean(scores))
                horizon_results[h] = {
                    'mean_rmse': mean_rmse_h,
                    'fold_scores': fold_results,
                }
                overall_rmse_list.append(mean_rmse_h)

            overall_mean_rmse = float(np.mean(overall_rmse_list))
            print(
                f"Trial {trial.number} finished, "
                f"overall mean RMSE across horizons={overall_mean_rmse:.4f}\n"
            )

            self.fold_history.append({
                'trial': trial.number,
                'params': params,
                'horizon_results': horizon_results,
                'overall_mean_rmse': overall_mean_rmse,
            })

            return overall_mean_rmse

        return objective

    # ======================== 4. RUN OPTUNA ========================

    def run_optuna(self, n_trials=50):
        """
        Run Optuna to find shared hyperparameters for all horizons.

        Dùng MedianPruner để cắt trial tệ.
        """
        objective_fn = self.create_objective()

        study = optuna.create_study(
            direction='minimize',
            pruner=optuna.pruners.MedianPruner(
                n_warmup_steps=5  # number of steps before pruning is enabled
            )
        )
        study.optimize(objective_fn, n_trials=n_trials)
        self.best_params = study.best_params
        return study

    # ======================== 5. TRAIN A FINAL MODEL PER HORIZON ========================

    def train_final_models(self):
        """
        Train one final model per horizon on the entire development set
        using the best hyperparameters from Optuna.
        """
        if self.best_params is None:
            raise ValueError("You must run run_optuna() before train_final_models().")

        final_params = self.best_params.copy()
        final_params.update({
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'n_estimators': 800
        })

        X_full = self.df[self.X_cols]

        for h in self.horizons:
            y_full_h = self.df[f"{self.target_col}_h{h}"]

            model_h = xgb.XGBRegressor(
                **final_params,
                random_state=42,
                tree_method='hist'
            )

            model_h.fit(X_full, y_full_h, verbose=False)
            self.final_models[h] = model_h

    # ======================== 6. PREDICTION HELPERS ========================

    def predict_horizon(self, X_today, h):
        """
        Predict target at t+h for a single row or DataFrame X_today.
        """
        if h not in self.horizons:
            raise ValueError(f"Horizon {h} not in configured horizons {self.horizons}.")
        if self.final_models[h] is None:
            raise ValueError(
                f"Model for horizon {h} is not trained. "
                f"Call train_final_models() first."
            )

        X_input = X_today[self.X_cols]
        return self.final_models[h].predict(X_input)

    def predict_all_horizons(self, X_today):
        """
        Predict for all configured horizons for a given X_today (1 row or small DataFrame).
        Returns a dict: {h: prediction_array}
        """
        results = {}
        for h in self.horizons:
            results[h] = self.predict_horizon(X_today, h)
        return results

    # ======================== 6b. BUILD PREDICTIONS FRAME (Plot) ========================

    def get_predictions_frame(self, df_with_shift, h):
        """
        Build a DataFrame with dates, true values and predictions for horizon h.

        Parameters
        ----------
        df_with_shift : pd.DataFrame
            DataFrame (train hoặc test) đã có sẵn các cột:
            - self.date_col
            - f"{self.target_col}_h{h}"
            - f"{self.date_col}_h{h}" (nếu có)

        Returns
        -------
        df_pred : pd.DataFrame with columns:
            - feature_time : time t (where features are taken from)
            - target_time  : time t+h (if available), else feature_time
            - y_true       : true target at t+h
            - y_pred       : model prediction at t+h
        """
        if h not in self.horizons:
            raise ValueError(f"Horizon {h} not in configured horizons {self.horizons}.")
        if self.final_models[h] is None:
            raise ValueError(
                f"Model for horizon {h} is not trained. "
                f"Call train_final_models() first."
            )

        X = df_with_shift[self.X_cols]
        y_true = df_with_shift[f"{self.target_col}_h{h}"]
        y_pred = self.final_models[h].predict(X)

        feature_time = df_with_shift[self.date_col]
        if f"{self.date_col}_h{h}" in df_with_shift.columns:
            target_time = df_with_shift[f"{self.date_col}_h{h}"]
        else:
            # fallback: dùng feature_time nếu không có date_h
            target_time = feature_time

        df_pred = pd.DataFrame({
            "feature_time": feature_time.values,
            "target_time": target_time.values,
            "y_true": y_true.values,
            "y_pred": y_pred,
        })
        return df_pred

    def plot_predictions(self, df_with_shift, h, use_target_time=True, n_points=None):
        """
        Plot true vs predicted for a given horizon h.

        Parameters
        ----------
        df_with_shift : pd.DataFrame
            DataFrame (thường là test_df đã shift bằng prepare_test_dataset).
        h : int
            Horizon to plot (must be in self.horizons).
        use_target_time : bool, default True
            If True and target_time exists, x-axis = target_time (t+h).
            If False, x-axis = feature_time (t).
        n_points : int or None
            If not None, only the first n_points are plotted.
        """
        df_pred = self.get_predictions_frame(df_with_shift, h)

        df_plot = df_pred.copy()
        if use_target_time and "target_time" in df_plot.columns:
            df_plot["x"] = df_plot["target_time"]
        else:
            df_plot["x"] = df_plot["feature_time"]

        if n_points is not None:
            df_plot = df_plot.iloc[:n_points]

        plt.figure(figsize=(10, 4))
        plt.plot(df_plot["x"], df_plot["y_true"], label="True")
        plt.plot(df_plot["x"], df_plot["y_pred"], label="Predicted")
        plt.xlabel("Date")
        plt.ylabel(self.target_col)
        plt.title(f"Horizon h={h} forecast")
        plt.legend()
        plt.tight_layout()
        plt.show()

    # ======================== 7. EVAL ON TRAIN SET (CHECK OVERFIT) ========================

    def evaluate_train_models(self):
        """
        Evaluate all final models on the training (development) set.
        Returns a dict: {h: {'rmse','mae','mape','mse','r2'}}
        """
        if any(self.final_models[h] is None for h in self.horizons):
            raise ValueError("All horizon models must be trained before evaluation.")

        metrics = {}
        X_train_full = self.df[self.X_cols]

        for h in self.horizons:
            y_train_h = self.df[f"{self.target_col}_h{h}"]
            preds = self.final_models[h].predict(X_train_full)

            rmse = mean_squared_error(y_train_h, preds)
            mae = mean_absolute_error(y_train_h, preds)
            mse = mean_squared_error(y_train_h, preds)

            # avoid division by zero for MAPE
            eps = 1e-8
            mape = np.mean(np.abs((y_train_h - preds) / (y_train_h + eps))) * 100

            ss_res = np.sum((y_train_h - preds) ** 2)
            ss_tot = np.sum((y_train_h - np.mean(y_train_h)) ** 2)
            r2 = 1 - ss_res / ss_tot if ss_tot != 0 else 0.0

            print(
                f"[TRAIN] Horizon h={h} -> "
                f"RMSE: {rmse:.4f}, R²: {r2:.4f}"
            )

            metrics[h] = {
                'rmse': rmse,
                'mae': mae,
                'mape': mape,
                'mse': mse,
                'r2': r2,
            }

        return metrics

    # ======================== 8. EVALUATION ON A TEST SET ========================

    def evaluate_final_models(self, test_df):
        """
        Evaluate all final models on a separate test DataFrame that already contains
        the shifted targets (i.e., you must have created _h1.._h5 on it).
        Returns a dict: {h: {'rmse','mae','mape','mse','r2'}}
        """
        if any(self.final_models[h] is None for h in self.horizons):
            raise ValueError("All horizon models must be trained before evaluation.")

        metrics = {}
        X_test = test_df[self.X_cols]

        for h in self.horizons:
            y_test_h = test_df[f"{self.target_col}_h{h}"]
            preds = self.final_models[h].predict(X_test)

            rmse = mean_squared_error(y_test_h, preds)
            mae = mean_absolute_error(y_test_h, preds)
            mse = mean_squared_error(y_test_h, preds)

            eps = 1e-8
            mape = np.mean(np.abs((y_test_h - preds) / (y_test_h + eps))) * 100

            ss_res = np.sum((y_test_h - preds) ** 2)
            ss_tot = np.sum((y_test_h - np.mean(y_test_h)) ** 2)
            r2 = 1 - ss_res / ss_tot if ss_tot != 0 else 0.0

            print(f"Horizon h={h} -> RMSE: {rmse:.4f}, R²: {r2:.4f}")

            metrics[h] = {
                'rmse': rmse,
                'mae': mae,
                'mape': mape,
                'mse': mse,
                'r2': r2,
            }

        return metrics

In [ ]:
## model_training 2

In [6]:
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import json  # Vẫn cần cho file tóm tắt (summary)
import datetime  # Vẫn cần cho file tóm tắt (summary)
import pickle  # Dùng để lưu model .pkl


class NpEncoder(json.JSONEncoder):
    """
    Custom JSON encoder để xử lý các kiểu dữ liệu của NumPy
    (vd: np.float64, np.int64) mà thư viện json tiêu chuẩn không xử lý được.
    """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)


class MultiHorizonWalkForwardOptuna_XGBoost_Pipeline:
    def __init__(
        self,
        df,
        date_col,
        target_col,
        feature_cols,
        n_splits=5,
        test_size=30,
        mode="expanding",
        horizons=(1, 2, 3, 4, 5),  # t+1 ... t+5
    ):
        """
        ... (docstring) ...
        """
        self.df = df.copy().sort_values(date_col).reset_index(drop=True)
        self.date_col = date_col
        self.target_col = target_col
        self.feature_cols = feature_cols
        self.n_splits = n_splits
        self.test_size = test_size
        self.mode = mode
        self.horizons = tuple(horizons)

        self.walkfolds = {h: [] for h in self.horizons}
        self.walkfold_dates = {h: [] for h in self.horizons}

        self.final_models = {h: None for h in self.horizons}

        self.best_params = None
        self.fold_history = []

        self.X_cols = [
            c for c in self.feature_cols
            if c not in [
                self.date_col,
                self.target_col,
                self.target_col + "_next",
                self.date_col + "_next",
            ]
        ]

    # ======================== 1. TARGET SHIFT FOR ALL HORIZONS (TRAIN) ========================

    def add_target_shifts(self):
        """
        ... (docstring) ...
        """
        for h in self.horizons:
            self.df[f"{self.target_col}_h{h}"] = self.df[self.target_col].shift(-h)
            self.df[f"{self.date_col}_h{h}"] = self.df[self.date_col].shift(-h)

        target_cols = [f"{self.target_col}_h{h}" for h in self.horizons]
        self.df = self.df.dropna(subset=target_cols).reset_index(drop=True)

    # ======================== 1b. TARGET SHIFT CHO TEST SET ========================

    def prepare_test_dataset(self, test_df_raw):
        """
        ... (docstring) ...
        """
        test_df = test_df_raw.copy().sort_values(self.date_col).reset_index(drop=True)

        for h in self.horizons:
            test_df[f"{self.target_col}_h{h}"] = test_df[self.target_col].shift(-h)
            test_df[f"{self.date_col}_h{h}"] = test_df[self.date_col].shift(-h)

        target_cols = [f"{self.target_col}_h{h}" for h in self.horizons]
        test_df = test_df.dropna(subset=target_cols).reset_index(drop=True)
        return test_df

    # ======================== 2. CREATE WALK-FORWARD FOLDS ========================

    def create_walkforward_folds(self):
        """
        ... (docstring) ...
        """
        df_len = len(self.df)
        step = (df_len - self.test_size) // self.n_splits

        self.walkfolds = {h: [] for h in self.horizons}
        self.walkfold_dates = {h: [] for h in self.horizons}

        for i in range(self.n_splits):
            train_end = step * (i + 1)
            test_start = train_end + 91  # 91-day gap
            test_end = test_start + self.test_size
            if test_end > df_len:
                break

            if self.mode == "expanding":
                train_start = 0
            else:  # rolling
                train_start = max(0, train_end - step * 2)

            train_df = self.df.iloc[train_start:train_end]
            test_df = self.df.iloc[test_start:test_end]

            X_train = train_df[self.X_cols].reset_index(drop=True)
            X_val = test_df[self.X_cols].reset_index(drop=True)

            X_train_dates = train_df[self.date_col].reset_index(drop=True)
            X_val_dates = test_df[self.date_col].reset_index(drop=True)

            for h in self.horizons:
                y_train_h = train_df[f"{self.target_col}_h{h}"].reset_index(drop=True)
                y_val_h = test_df[f"{self.target_col}_h{h}"].reset_index(drop=True)

                y_train_dates_h = train_df[f"{self.date_col}_h{h}"].reset_index(drop=True)
                y_val_dates_h = test_df[f"{self.date_col}_h{h}"].reset_index(drop=True)

                self.walkfolds[h].append(
                    ((X_train, y_train_h), (X_val, y_val_h))
                )

                self.walkfold_dates[h].append(
                    ((X_train_dates, y_train_dates_h),
                     (X_val_dates, y_val_dates_h))
                )

    # ======================== 3. OPTUNA OBJECTIVE (SHARED PARAMS) ========================

    def create_objective(self):
        """
        ... (docstring) ...
        """
        def objective(trial):
            params = {
                'objective': 'reg:squarederror',
                'eval_metric': 'rmse',
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'max_depth': trial.suggest_int('max_depth', 3, 6),
                'gamma': trial.suggest_float('gamma', 0.0, 1.0),
                'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 10.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
                'subsample': trial.suggest_float('subsample', 0.5, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'min_child_weight': trial.suggest_int('min_child_weight', 3, 10),
            }

            horizon_results = {}
            overall_rmse_list = []
            global_step = 0

            for h in self.horizons:
                scores = []
                fold_results = []

                for i, ((X_train, y_train), (X_val, y_val)) in enumerate(self.walkfolds[h]):

                    model = xgb.XGBRegressor(
                        **params,
                        n_estimators=800,
                        random_state=42,
                        tree_method='hist',
                        early_stopping_rounds=100
                    )

                    model.fit(
                        X_train,
                        y_train,
                        eval_set=[(X_val, y_val)],
                        verbose=False
                    )

                    preds = model.predict(X_val)
                    
                    # ĐÃ SỬA (FIXED): Lấy căn bậc hai của MSE để ra RMSE
                    rmse = np.sqrt(mean_squared_error(y_val, preds))

                    ss_res = np.sum((y_val - preds) ** 2)
                    ss_tot = np.sum((y_val - np.mean(y_val)) ** 2)
                    r2 = 1 - ss_res / ss_tot if ss_tot != 0 else 0.0

                    scores.append(rmse)
                    fold_results.append({'rmse': rmse, 'r2': r2})

                    print(
                        f"Trial {trial.number}, Horizon h={h}, "
                        f"Fold {i+1}: RMSE={rmse:.4f}, R²={r2:.4f}"
                    )

                    trial.report(rmse, step=global_step)
                    if trial.should_prune():
                        print(f"Trial {trial.number} pruned at global_step={global_step}")
                        raise optuna.TrialPruned()
                    global_step += 1

                mean_rmse_h = float(np.mean(scores))
                horizon_results[h] = {
                    'mean_rmse': mean_rmse_h,
                    'fold_scores': fold_results,
                }
                overall_rmse_list.append(mean_rmse_h)

            overall_mean_rmse = float(np.mean(overall_rmse_list))
            print(
                f"Trial {trial.number} finished, "
                f"overall mean RMSE across horizons={overall_mean_rmse:.4f}\n"
            )

            self.fold_history.append({
                'trial': trial.number,
                'params': params,
                'horizon_results': horizon_results,
                'overall_mean_rmse': overall_mean_rmse,
            })

            return overall_mean_rmse

        return objective

    # ======================== 4. RUN OPTUNA ========================

    def run_optuna(self, n_trials=50):
        """
        ... (docstring) ...
        """
        objective_fn = self.create_objective()

        study = optuna.create_study(
            direction='minimize',
            pruner=optuna.pruners.MedianPruner(
                n_warmup_steps=5
            )
        )
        study.optimize(objective_fn, n_trials=n_trials)
        self.best_params = study.best_params
        return study

    # ======================== 5. TRAIN A FINAL MODEL PER HORIZON ========================

    def train_final_models(self):
        """
        ... (docstring) ...
        """
        if self.best_params is None:
            raise ValueError("You must run run_optuna() before train_final_models().")

        final_params = self.best_params.copy()
        final_params.update({
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'n_estimators': 800
        })

        X_full = self.df[self.X_cols]

        for h in self.horizons:
            y_full_h = self.df[f"{self.target_col}_h{h}"]

            model_h = xgb.XGBRegressor(
                **final_params,
                random_state=42,
                tree_method='hist'
            )

            model_h.fit(X_full, y_full_h, verbose=False)
            self.final_models[h] = model_h
            print(f"Final model for horizon {h} trained.")

    # ======================== 6. PREDICTION HELPERS ========================

    def predict_horizon(self, X_today, h):
        """
        ... (docstring) ...
        """
        if h not in self.horizons:
            raise ValueError(f"Horizon {h} not in configured horizons {self.horizons}.")
        if self.final_models[h] is None:
            raise ValueError(f"Model for horizon {h} is not trained. Call train_final_models() first.")

        X_input = X_today[self.X_cols]
        return self.final_models[h].predict(X_input)

    def predict_all_horizons(self, X_today):
        """
        ... (docstring) ...
        """
        results = {}
        for h in self.horizons:
            results[h] = self.predict_horizon(X_today, h)
        return results

    # ======================== 6b. BUILD PREDICTIONS FRAME (Plot) ========================

    def get_predictions_frame(self, df_with_shift, h):
        """
        ... (docstring) ...
        """
        if h not in self.horizons:
            raise ValueError(f"Horizon {h} not in configured horizons {self.horizons}.")
        if self.final_models[h] is None:
            raise ValueError(f"Model for horizon {h} is not trained. Call train_final_models() first.")

        X = df_with_shift[self.X_cols]
        y_true = df_with_shift[f"{self.target_col}_h{h}"]
        y_pred = self.final_models[h].predict(X)

        feature_time = df_with_shift[self.date_col]
        if f"{self.date_col}_h{h}" in df_with_shift.columns:
            target_time = df_with_shift[f"{self.date_col}_h{h}"]
        else:
            target_time = feature_time

        df_pred = pd.DataFrame({
            "feature_time": feature_time.values,
            "target_time": target_time.values,
            "y_true": y_true.values,
            "y_pred": y_pred,
        })
        return df_pred

    def plot_predictions(self, df_with_shift, h, use_target_time=True, n_points=None):
        """
        ... (docstring) ...
        """
        df_pred = self.get_predictions_frame(df_with_shift, h)

        df_plot = df_pred.copy()
        if use_target_time and "target_time" in df_plot.columns:
            df_plot["x"] = df_plot["target_time"]
        else:
            df_plot["x"] = df_plot["feature_time"]

        if n_points is not None:
            df_plot = df_plot.iloc[:n_points]

        plt.figure(figsize=(10, 4))
        plt.plot(df_plot["x"], df_plot["y_true"], label="True")
        plt.plot(df_plot["x"], df_plot["y_pred"], label="Predicted")
        plt.xlabel("Date")
        plt.ylabel(self.target_col)
        plt.title(f"Horizon h={h} forecast")
        plt.legend()
        plt.tight_layout()
        plt.show()

    # ======================== 7. EVAL ON TRAIN SET (CHECK OVERFIT) ========================

    def evaluate_train_models(self):
        """
        ... (docstring) ...
        """
        if any(self.final_models[h] is None for h in self.horizons):
            raise ValueError("All horizon models must be trained before evaluation.")

        metrics = {}
        X_train_full = self.df[self.X_cols]

        for h in self.horizons:
            y_train_h = self.df[f"{self.target_col}_h{h}"]
            preds = self.final_models[h].predict(X_train_full)

            # ĐÃ SỬA (FIXED): Lấy căn bậc hai của MSE
            mse = mean_squared_error(y_train_h, preds)
            rmse = np.sqrt(mse)
            
            mae = mean_absolute_error(y_train_h, preds)

            eps = 1e-8
            mape = np.mean(np.abs((y_train_h - preds) / (y_train_h + eps))) * 100

            ss_res = np.sum((y_train_h - preds) ** 2)
            ss_tot = np.sum((y_train_h - np.mean(y_train_h)) ** 2)
            r2 = 1 - ss_res / ss_tot if ss_tot != 0 else 0.0

            print(
                f"[TRAIN] Horizon h={h} -> "
                f"RMSE: {rmse:.4f}, R²: {r2:.4f}"
            )

            metrics[h] = {
                'rmse': rmse,
                'mae': mae,
                'mape': mape,
                'mse': mse,
                'r2': r2,
            }

        return metrics

    # ======================== 8. EVALUATION ON A TEST SET ========================

    def evaluate_final_models(self, test_df):
        """
        ... (docstring) ...
        """
        if any(self.final_models[h] is None for h in self.horizons):
            raise ValueError("All horizon models must be trained before evaluation.")

        metrics = {}
        X_test = test_df[self.X_cols]

        for h in self.horizons:
            y_test_h = test_df[f"{self.target_col}_h{h}"]
            preds = self.final_models[h].predict(X_test)

            # ĐÃ SỬA (FIXED): Lấy căn bậc hai của MSE
            mse = mean_squared_error(y_test_h, preds)
            rmse = np.sqrt(mse)

            mae = mean_absolute_error(y_test_h, preds)

            eps = 1e-8
            mape = np.mean(np.abs((y_test_h - preds) / (y_test_h + eps))) * 100

            ss_res = np.sum((y_test_h - preds) ** 2)
            ss_tot = np.sum((y_test_h - np.mean(y_test_h)) ** 2)
            r2 = 1 - ss_res / ss_tot if ss_tot != 0 else 0.0

            print(f"Horizon h={h} -> RMSE: {rmse:.4f}, R²: {r2:.4f}")

            metrics[h] = {
                'rmse': rmse,
                'mae': mae,
                'mape': mape,
                'mse': mse,
                'r2': r2,
            }

        return metrics

    # ======================== 9. SAVE FINAL MODELS (PICKLE) ========================

    def save_all_final_models_pkl(self, base_filename_prefix):
        """
        Lưu tất cả các model cuối cùng (final_models) ra đĩa dùng PICKLE (.pkl).
        """
        if any(self.final_models[h] is None for h in self.horizons):
            raise ValueError("Phải train model trước khi lưu. Hãy gọi train_final_models().")
        
        saved_paths = {}
        for h in self.horizons:
            model = self.final_models[h]
            filename = f"{base_filename_prefix}_h{h}.pkl"
            
            try:
                with open(filename, 'wb') as f: 
                    pickle.dump(model, f)
                saved_paths[h] = filename
                print(f"Model (Pickle) cho horizon {h} đã được lưu vào: {filename}")
            except Exception as e:
                print(f"Lỗi khi lưu file pickle {filename}: {e}")
        
        return saved_paths

    # ======================== 10. SAVE RESULTS SUMMARY (JSON) ========================
    
    def save_results_to_json(
        self,
        filename,
        model_name,
        train_metrics,
        test_metrics,
        saved_model_paths=None
    ):
        """
        Lưu kết quả tóm tắt (best param, metrics, timestamp,...) vào 1 file JSON.
        """
        
        results_summary = {
            "model_name": model_name,
            "run_timestamp_utc": datetime.utcnow().isoformat(),
            "target_column": self.target_col,
            "feature_columns": self.X_cols,
            "horizons": self.horizons,
            "cross_validation_setup": {
                "n_splits": self.n_splits,
                "test_size_per_fold": self.test_size,
                "mode": self.mode,
            },
            "optuna_best_params": self.best_params,
            "evaluation_train_set": train_metrics,
            "evaluation_test_set": test_metrics,
            "saved_model_files": saved_model_paths if saved_model_paths else {},
            "optuna_fold_history": self.fold_history 
        }

        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(results_summary, f, indent=4, cls=NpEncoder)
            print(f"Kết quả tóm tắt đã được lưu vào: {filename}")
        except Exception as e:
            print(f"Lỗi khi lưu file JSON: {e}")

c:\Users\ASUS\anaconda3\envs\wp_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
XGBoost_pipeline = MultiHorizonWalkForwardOptuna_XGBoost_Pipeline(
    df=train_dataset,
    date_col="datetime",
    target_col="temp",
    feature_cols=[col for col in train_dataset.columns if col not in ["datetime", "temp", "temp_next", "datetime_next"]],
    n_splits=5,
    test_size=365,
    mode="expanding",
    horizons=(1, 2, 3, 4, 5)
)

In [8]:
XGBoost_pipeline.add_target_shifts()
XGBoost_pipeline.create_walkforward_folds()


In [9]:
XGBoost_pipeline.run_optuna(n_trials=100)

[I 2025-11-17 02:07:08,615] A new study created in memory with name: no-name-f405f82f-5c1e-469e-aac9-e0905409c477


Trial 0, Horizon h=1, Fold 1: RMSE=1.8533, R²=0.8209
Trial 0, Horizon h=1, Fold 2: RMSE=1.8492, R²=0.8473
Trial 0, Horizon h=1, Fold 3: RMSE=1.6713, R²=0.8907
Trial 0, Horizon h=1, Fold 4: RMSE=1.6918, R²=0.8988
Trial 0, Horizon h=2, Fold 1: RMSE=2.2235, R²=0.7422
Trial 0, Horizon h=2, Fold 2: RMSE=2.4222, R²=0.7371
Trial 0, Horizon h=2, Fold 3: RMSE=2.2677, R²=0.7994
Trial 0, Horizon h=2, Fold 4: RMSE=2.2820, R²=0.8162
Trial 0, Horizon h=3, Fold 1: RMSE=2.3913, R²=0.7020
Trial 0, Horizon h=3, Fold 2: RMSE=2.6792, R²=0.6773
Trial 0, Horizon h=3, Fold 3: RMSE=2.5775, R²=0.7408
Trial 0, Horizon h=3, Fold 4: RMSE=2.4817, R²=0.7827
Trial 0, Horizon h=4, Fold 1: RMSE=2.3933, R²=0.7017
Trial 0, Horizon h=4, Fold 2: RMSE=2.7426, R²=0.6609
Trial 0, Horizon h=4, Fold 3: RMSE=2.6549, R²=0.7253
Trial 0, Horizon h=4, Fold 4: RMSE=2.5902, R²=0.7633
Trial 0, Horizon h=5, Fold 1: RMSE=2.4525, R²=0.6871
Trial 0, Horizon h=5, Fold 2: RMSE=2.7724, R²=0.6529
Trial 0, Horizon h=5, Fold 3: RMSE=2.6964, R²=

[I 2025-11-17 02:07:31,497] Trial 0 finished with value: 2.36398679654312 and parameters: {'learning_rate': 0.019816715189721303, 'max_depth': 3, 'gamma': 0.5684681137445113, 'reg_lambda': 7.183245735862915, 'reg_alpha': 5.003389999588296, 'subsample': 0.8471590121169105, 'colsample_bytree': 0.9253241540238196, 'min_child_weight': 7}. Best is trial 0 with value: 2.36398679654312.


Trial 0, Horizon h=5, Fold 4: RMSE=2.5868, R²=0.7643
Trial 0 finished, overall mean RMSE across horizons=2.3640

Trial 1, Horizon h=1, Fold 1: RMSE=1.8390, R²=0.8237
Trial 1, Horizon h=1, Fold 2: RMSE=1.8309, R²=0.8503
Trial 1, Horizon h=1, Fold 3: RMSE=1.6877, R²=0.8886
Trial 1, Horizon h=1, Fold 4: RMSE=1.7393, R²=0.8930
Trial 1, Horizon h=2, Fold 1: RMSE=2.2624, R²=0.7331
Trial 1, Horizon h=2, Fold 2: RMSE=2.4444, R²=0.7323
Trial 1, Horizon h=2, Fold 3: RMSE=2.3145, R²=0.7911
Trial 1, Horizon h=2, Fold 4: RMSE=2.2987, R²=0.8135
Trial 1, Horizon h=3, Fold 1: RMSE=2.3789, R²=0.7051
Trial 1, Horizon h=3, Fold 2: RMSE=2.6747, R²=0.6784
Trial 1, Horizon h=3, Fold 3: RMSE=2.5639, R²=0.7436
Trial 1, Horizon h=3, Fold 4: RMSE=2.5178, R²=0.7763
Trial 1, Horizon h=4, Fold 1: RMSE=2.4138, R²=0.6966
Trial 1, Horizon h=4, Fold 2: RMSE=2.7363, R²=0.6624
Trial 1, Horizon h=4, Fold 3: RMSE=2.6407, R²=0.7282
Trial 1, Horizon h=4, Fold 4: RMSE=2.5932, R²=0.7628
Trial 1, Horizon h=5, Fold 1: RMSE=2.51

[I 2025-11-17 02:07:52,835] Trial 1 finished with value: 2.3783439985222343 and parameters: {'learning_rate': 0.02884504358693142, 'max_depth': 4, 'gamma': 0.8239988667527282, 'reg_lambda': 7.601689398190267, 'reg_alpha': 5.02550302936182, 'subsample': 0.7407092294539734, 'colsample_bytree': 0.6561406781710444, 'min_child_weight': 7}. Best is trial 0 with value: 2.36398679654312.


...

Trial 97, Horizon h=5, Fold 4: RMSE=2.6023, R²=0.7615
Trial 97 finished, overall mean RMSE across horizons=2.3394

Trial 98, Horizon h=1, Fold 1: RMSE=1.7993, R²=0.8312
Trial 98, Horizon h=1, Fold 2: RMSE=1.8158, R²=0.8528
Trial 98, Horizon h=1, Fold 3: RMSE=1.6310, R²=0.8959
Trial 98, Horizon h=1, Fold 4: RMSE=1.6443, R²=0.9044
Trial 98, Horizon h=2, Fold 1: RMSE=2.1964, R²=0.7484
Trial 98, Horizon h=2, Fold 2: RMSE=2.4009, R²=0.7417
Trial 98, Horizon h=2, Fold 3: RMSE=2.2414, R²=0.8041
Trial 98, Horizon h=2, Fold 4: RMSE=2.2529, R²=0.8209
Trial 98, Horizon h=3, Fold 1: RMSE=2.3782, R²=0.7052
Trial 98, Horizon h=3, Fold 2: RMSE=2.6233, R²=0.6906
Trial 98, Horizon h=3, Fold 3: RMSE=2.4922, R²=0.7577
Trial 98, Horizon h=3, Fold 4: RMSE=2.4327, R²=0.7912
Trial 98, Horizon h=4, Fold 1: RMSE=2.3793, R²=0.7052
Trial 98, Horizon h=4, Fold 2: RMSE=2.7303, R²=0.6639
Trial 98, Horizon h=4, Fold 3: RMSE=2.5836, R²=0.7398
Trial 98, Horizon h=4, Fold 4: RMSE=2.5501, R²=0.7706
Trial 98, Horizon h=5

[I 2025-11-17 02:36:44,248] Trial 98 finished with value: 2.33064448496772 and parameters: {'learning_rate': 0.024948035864040503, 'max_depth': 3, 'gamma': 0.5125985045484892, 'reg_lambda': 7.033856515113431, 'reg_alpha': 0.577011713671673, 'subsample': 0.5747850121407979, 'colsample_bytree': 0.5231067339158282, 'min_child_weight': 10}. Best is trial 67 with value: 2.33043015703495.


Trial 98, Horizon h=5, Fold 4: RMSE=2.6357, R²=0.7553
Trial 98 finished, overall mean RMSE across horizons=2.3306

Trial 99, Horizon h=1, Fold 1: RMSE=1.8058, R²=0.8300
Trial 99, Horizon h=1, Fold 2: RMSE=1.8269, R²=0.8510
Trial 99, Horizon h=1, Fold 3: RMSE=1.6333, R²=0.8957
Trial 99, Horizon h=1, Fold 4: RMSE=1.6813, R²=0.9000
Trial 99, Horizon h=2, Fold 1: RMSE=2.2040, R²=0.7467
Trial 99, Horizon h=2, Fold 2: RMSE=2.4167, R²=0.7383
Trial 99, Horizon h=2, Fold 3: RMSE=2.2348, R²=0.8052
Trial 99, Horizon h=2, Fold 4: RMSE=2.2603, R²=0.8197
Trial 99, Horizon h=3, Fold 1: RMSE=2.3826, R²=0.7042
Trial 99, Horizon h=3, Fold 2: RMSE=2.6233, R²=0.6906
Trial 99, Horizon h=3, Fold 3: RMSE=2.4770, R²=0.7606
Trial 99, Horizon h=3, Fold 4: RMSE=2.4725, R²=0.7843
Trial 99, Horizon h=4, Fold 1: RMSE=2.3753, R²=0.7062
Trial 99, Horizon h=4, Fold 2: RMSE=2.7155, R²=0.6675
Trial 99, Horizon h=4, Fold 3: RMSE=2.5645, R²=0.7437
Trial 99, Horizon h=4, Fold 4: RMSE=2.5756, R²=0.7660
Trial 99, Horizon h=5

[I 2025-11-17 02:36:56,444] Trial 99 finished with value: 2.3335175586943286 and parameters: {'learning_rate': 0.031227524168160357, 'max_depth': 3, 'gamma': 0.48350061617124873, 'reg_lambda': 7.181124352900384, 'reg_alpha': 1.0773652638902793, 'subsample': 0.5744024998598926, 'colsample_bytree': 0.525020478654193, 'min_child_weight': 10}. Best is trial 67 with value: 2.33043015703495.


Trial 99, Horizon h=5, Fold 4: RMSE=2.6092, R²=0.7602
Trial 99 finished, overall mean RMSE across horizons=2.3335



In [10]:
XGBoost_pipeline.train_final_models()

Final model for horizon 1 trained.
Final model for horizon 2 trained.
Final model for horizon 3 trained.
Final model for horizon 4 trained.
Final model for horizon 5 trained.


In [11]:
# Train metrics
train_metrics = XGBoost_pipeline.evaluate_train_models()
train_metrics

[TRAIN] Horizon h=1 -> RMSE: 1.0788, R²: 0.9553
[TRAIN] Horizon h=2 -> RMSE: 1.4325, R²: 0.9213
[TRAIN] Horizon h=3 -> RMSE: 1.5088, R²: 0.9128
[TRAIN] Horizon h=4 -> RMSE: 1.5107, R²: 0.9128
[TRAIN] Horizon h=5 -> RMSE: 1.5134, R²: 0.9125


{1: {'rmse': np.float64(1.0788379517976316),
  'mae': 0.8373182903759213,
  'mape': np.float64(3.591224370775793),
  'mse': 1.1638913262389088,
  'r2': np.float64(0.9553090657523808)},
 2: {'rmse': np.float64(1.4325019023468066),
  'mae': 1.1142996782629293,
  'mape': np.float64(4.8453293980888485),
  'mse': 2.05206170022722,
  'r2': np.float64(0.9213064448620136)},
 3: {'rmse': np.float64(1.5088288048681475),
  'mae': 1.1944102565166468,
  'mape': np.float64(5.219128309926884),
  'mse': 2.2765643623998426,
  'r2': np.float64(0.912840717041869)},
 4: {'rmse': np.float64(1.5106953746848146),
  'mae': 1.2069675357203504,
  'mape': np.float64(5.271946234517428),
  'mse': 2.2822005150940923,
  'r2': np.float64(0.91275122057154)},
 5: {'rmse': np.float64(1.5133685685481173),
  'mae': 1.2018135133524763,
  'mape': np.float64(5.246266050264323),
  'mse': 2.2902844242693776,
  'r2': np.float64(0.9125448443356914)}}

In [12]:
# Test metrics
test_df_shifted = XGBoost_pipeline.prepare_test_dataset(test_dataset)
test_metrics = XGBoost_pipeline.evaluate_final_models(test_df_shifted)

Horizon h=1 -> RMSE: 1.6162, R²: 0.8925
Horizon h=2 -> RMSE: 2.1263, R²: 0.8137
Horizon h=3 -> RMSE: 2.3462, R²: 0.7731
Horizon h=4 -> RMSE: 2.4071, R²: 0.7612
Horizon h=5 -> RMSE: 2.4587, R²: 0.7509


In [24]:
pd.DataFrame(test_metrics)

,1,2,3,4,5
rmse,1.468970,2.046809,2.295129,2.390412,2.441394
mae,1.155537,1.612771,1.822410,1.883491,1.912807
mape,4.901593,6.924236,7.886138,8.165083,8.298942
mse,2.157872,4.189429,5.267615,5.714070,5.960403
r2,0.911156,0.827366,0.782903,0.764505,0.754355


In [13]:
# (Giả sử bạn đã import "from pathlib import Path" ở đầu file)
# (Và "PROJECT_ROOT" đã được định nghĩa như code của bạn)
from datetime import datetime


In [14]:

# --- 10. LƯU KẾT QUẢ (Phiên bản cập nhật có timestamp) ---

# 0. Tạo timestamp theo thời gian chạy
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_name = f"hanoi_temp_v1_{timestamp}"

# 1. Tái sử dụng biến PROJECT_ROOT để tạo đường dẫn lưu
BASE_SAVE_DIR = Path(PROJECT_ROOT) / "src" / "config"

# 2. Tạo thư mục con
MODELS_DIR = BASE_SAVE_DIR / "models_pkl"
RESULTS_DIR = BASE_SAVE_DIR / "results"

# 3. Tự động tạo thư mục nếu chưa tồn tại
MODELS_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print(f"\nLưu các model (file .pkl) vào: {MODELS_DIR}")

# 4. Lưu file .pkl theo format có timestamp (không ghi đè file cũ)
model_paths_pkl = XGBoost_pipeline.save_all_final_models_pkl(
    base_filename_prefix=str(MODELS_DIR / run_name)
)

print(f"Lưu file tóm tắt (file .json) vào: {RESULTS_DIR}")

# 5. Lưu JSON có timestamp
XGBoost_pipeline.save_results_to_json(
    filename=str(RESULTS_DIR / f"{run_name}_summary.json"),
    model_name="Hanoi Temp Predictor v1.0 (XGBoost)",
    train_metrics=train_metrics,
    test_metrics=test_metrics,
    saved_model_paths=model_paths_pkl
)

print("\n--- HOÀN TẤT TOÀN BỘ FLOW ---")
print(f"Run name: {run_name}")


Lưu các model (file .pkl) vào: C:\Assigment\2025-2026\Machine Learning\machine_learning_project\src\config\models_pkl
Model (Pickle) cho horizon 1 đã được lưu vào: C:\Assigment\2025-2026\Machine Learning\machine_learning_project\src\config\models_pkl\hanoi_temp_v1_20251117_024235_h1.pkl
Model (Pickle) cho horizon 2 đã được lưu vào: C:\Assigment\2025-2026\Machine Learning\machine_learning_project\src\config\models_pkl\hanoi_temp_v1_20251117_024235_h2.pkl
Model (Pickle) cho horizon 3 đã được lưu vào: C:\Assigment\2025-2026\Machine Learning\machine_learning_project\src\config\models_pkl\hanoi_temp_v1_20251117_024235_h3.pkl
Model (Pickle) cho horizon 4 đã được lưu vào: C:\Assigment\2025-2026\Machine Learning\machine_learning_project\src\config\models_pkl\hanoi_temp_v1_20251117_024235_h4.pkl
Model (Pickle) cho horizon 5 đã được lưu vào: C:\Assigment\2025-2026\Machine Learning\machine_learning_project\src\config\models_pkl\hanoi_temp_v1_20251117_024235_h5.pkl
Lưu file tóm tắt (file .json) v

# Hourly

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import json
from datetime import datetime # Sửa lại import
import pickle

# ======================== JSON ENCODER (Giữ nguyên) ========================
class NpEncoder(json.JSONEncoder):
    """
    Custom JSON encoder để xử lý các kiểu dữ liệu của NumPy
    (vd: np.float64, np.int64) mà thư viện json tiêu chuẩn không xử lý được.
    """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

# ======================== HOURLY PIPELINE CLASS ========================

class MultiHorizonHourly_WalkForwardOptuna_XGBoost_Pipeline:
    """
    Pipeline chuyên dụng cho dự báo ĐA TẦM NHÌN (Multi-Horizon) HÀNG GIỜ.

    Sử dụng Walk-Forward Cross-Validation (với cửa sổ Mở rộng hoặc Trượt)
    kết hợp Optuna để tìm MỘT bộ siêu tham số (shared parameters) tối ưu
    cho tất cả các tầm nhìn (horizons).

    Sau đó, huấn luyện MỘT MÔ HÌNH CUỐI CÙNG (final model) riêng biệt
    cho TỪNG TẦM NHÌN (ví dụ: model cho t+1h, model cho t+3h,...).
    """
    
    def __init__(
        self,
        df,
        date_col,
        target_col,
        feature_cols,
        n_splits=5,
        test_size=720,       # THAY ĐỔI: Mặc định 720 giờ (30 ngày)
        gap_size_hours=24,    # THAY ĐỔI: Thêm Gap (giờ)
        mode="expanding",
        horizons=(1, 2, 3, 4 , 5), # THAY ĐỔI: Mặc định (giờ)
    ):
        """
        Khởi tạo pipeline.
        Args:
            df (pd.DataFrame): DataFrame chứa toàn bộ dữ liệu (đã qua FE).
            date_col (str): Tên cột datetime (đã sort).
            target_col (str): Tên cột mục tiêu (ví dụ: 'temp').
            feature_cols (list): Danh sách tất cả các cột feature.
            n_splits (int): Số lượng Fold cho Walk-Forward CV.
            test_size (int): Kích thước (số giờ) của mỗi tập test/validation.
            gap_size_hours (int): Số giờ chừa trống (gap) giữa train và test.
            mode (str): 'expanding' (cửa sổ mở rộng) hoặc 'rolling' (cửa sổ trượt).
            horizons (tuple): Các tầm nhìn dự đoán (ví dụ: t+1h, t+3h...).
        """
        self.df = df.copy().sort_values(date_col).reset_index(drop=True)
        self.date_col = date_col
        self.target_col = target_col
        self.feature_cols = feature_cols
        self.n_splits = n_splits
        self.test_size = test_size
        self.gap_size_hours = gap_size_hours # Đã thêm
        self.mode = mode
        self.horizons = tuple(horizons)

        self.walkfolds = {h: [] for h in self.horizons}
        self.walkfold_dates = {h: [] for h in self.horizons}

        self.final_models = {h: None for h in self.horizons}

        self.best_params = None
        self.fold_history = [] # Lưu lịch sử các fold của Optuna

        # Tự động lọc ra các cột X (features)
        self.X_cols = [
            c for c in self.feature_cols
            if c not in [
                self.date_col,
                self.target_col,
                # Loại bỏ các cột target/date đã shift (nếu có)
                self.target_col + "_next",
                self.date_col + "_next",
            ]
        ]
        # Đảm bảo không có cột target_hX nào trong X_cols
        for h in self.horizons:
            target_h_name = f"{self.target_col}_h{h}"
            date_h_name = f"{self.date_col}_h{h}"
            if target_h_name in self.X_cols:
                self.X_cols.remove(target_h_name)
            if date_h_name in self.X_cols:
                self.X_cols.remove(date_h_name)

    # ======================== 1. TARGET SHIFT FOR ALL HORIZONS (TRAIN) ========================

    def add_target_shifts(self):
        """
        Tạo các cột target đã dịch chuyển (shifted) cho từng horizon.
        Ví dụ: 'temp_h1' (temp 1 giờ sau), 'temp_h24' (temp 24 giờ sau).
        
        Xóa các hàng NaN ở cuối DataFrame (không có target).
        """
        print("Đang tạo các cột target (shifted)...")
        for h in self.horizons:
            # Dịch chuyển target (giá trị tương lai) về hàng hiện tại
            self.df[f"{self.target_col}_h{h}"] = self.df[self.target_col].shift(-h)
            # Dịch chuyển cả date của target (để dễ kiểm tra)
            self.df[f"{self.date_col}_h{h}"] = self.df[self.date_col].shift(-h)

        # Xóa các hàng ở cuối mà không có đủ target (ví dụ: 24h cuối)
        target_cols = [f"{self.target_col}_h{h}" for h in self.horizons]
        self.df = self.df.dropna(subset=target_cols).reset_index(drop=True)
        print(f"Hình dạng DF sau khi shift và dropna: {self.df.shape}")

    # ======================== 1b. TARGET SHIFT CHO TEST SET ========================

    def prepare_test_dataset(self, test_df_raw):
        """
        Chuẩn bị (shift target) cho một DataFrame test (hold-out) bên ngoài.
        """
        test_df = test_df_raw.copy().sort_values(self.date_col).reset_index(drop=True)

        for h in self.horizons:
            test_df[f"{self.target_col}_h{h}"] = test_df[self.target_col].shift(-h)
            test_df[f"{self.date_col}_h{h}"] = test_df[self.date_col].shift(-h)

        target_cols = [f"{self.target_col}_h{h}" for h in self.horizons]
        test_df = test_df.dropna(subset=target_cols).reset_index(drop=True)
        return test_df

    # ======================== 2. CREATE WALK-FORWARD FOLDS ========================

    def create_walkforward_folds(self):
        """
        Tạo ra các fold (cặp Train/Validation) theo chiến lược Walk-Forward.
        Lưu các fold (data) và ngày (dates) vào self.walkfolds
        """
        df_len = len(self.df)
        
        # Tính toán kích thước của mỗi 'bước' huấn luyện
        total_test_gap_span = (self.test_size + self.gap_size_hours) * self.n_splits
        train_span = df_len - total_test_gap_span
        
        if train_span <= 0:
             raise ValueError(
                 f"Dữ liệu quá ngắn ({df_len} giờ) cho n_splits={self.n_splits}, "
                 f"test_size={self.test_size}, gap={self.gap_size_hours}"
             )
        
        # 'step' là kích thước của block huấn luyện *ban đầu*
        # và cũng là kích thước của mỗi "bước nhảy"
        step = train_span // self.n_splits
        if step <= 0:
            raise ValueError(
                f"Không thể tạo Folds. 'step' (kích thước train) = {step}. "
                f"Hãy giảm n_splits hoặc test_size."
            )

        print(f"Đang tạo {self.n_splits} folds, mỗi fold test_size={self.test_size}h, gap={self.gap_size_hours}h...")

        self.walkfolds = {h: [] for h in self.horizons}
        self.walkfold_dates = {h: [] for h in self.horizons}

        # KHÔNG DÙNG 'initial_train_end' nữa
        # initial_train_end = df_len - total_test_gap_span

        for i in range(self.n_splits):
            
            # ========== SỬA LOGIC Ở ĐÂY ==========
            # train_end tăng thêm 'step' sau mỗi lần lặp
            train_end = step * (i + 1)
            
            if self.mode == "expanding":
                train_start = 0
            else: # 'rolling'
                # Cửa sổ trượt sẽ có kích thước bằng 'step'
                train_start = max(0, train_end - step)
            # ======================================
            
            test_start = train_end + self.gap_size_hours
            test_end = test_start + self.test_size
            
            if test_end > df_len:
                print(f"Bỏ qua fold {i+1} vì test_end ({test_end}) vượt quá độ dài dữ liệu ({df_len})")
                break

            train_df = self.df.iloc[train_start:train_end]
            test_df = self.df.iloc[test_start:test_end]
            
            # ... (Phần còn lại của hàm giữ nguyên) ...
            
            if train_df.empty or test_df.empty:
                print(f"Bỏ qua fold {i+1} vì train hoặc test rỗng.")
                continue

            # Tách X (features) và Y (targets)
            X_train = train_df[self.X_cols].reset_index(drop=True)
            X_val = test_df[self.X_cols].reset_index(drop=True)

            X_train_dates = train_df[self.date_col].reset_index(drop=True)
            X_val_dates = test_df[self.date_col].reset_index(drop=True)

            # Tạo dữ liệu cho từng horizon
            for h in self.horizons:
                y_train_h = train_df[f"{self.target_col}_h{h}"].reset_index(drop=True)
                y_val_h = test_df[f"{self.target_col}_h{h}"].reset_index(drop=True)

                y_train_dates_h = train_df[f"{self.date_col}_h{h}"].reset_index(drop=True)
                y_val_dates_h = test_df[f"{self.date_col}_h{h}"].reset_index(drop=True)

                self.walkfolds[h].append(
                    ((X_train, y_train_h), (X_val, y_val_h))
                )

                self.walkfold_dates[h].append(
                    ((X_train_dates, y_train_dates_h),
                     (X_val_dates, y_val_dates_h))
                )
        print(f"Đã tạo thành công {len(self.walkfolds[self.horizons[0]])} folds.")

    # ======================== 3. OPTUNA OBJECTIVE (SHARED PARAMS) ========================

    def create_objective(self):
        """
        Tạo hàm objective cho Optuna.
        Hàm này sẽ chạy Walk-Forward CV cho TẤT CẢ CÁC HORIZON
        và trả về RMSE TRUNG BÌNH CHUNG (mean over all horizons)
        """
        def objective(trial):
            # Không gian tìm kiếm siêu tham số
            params = {
                'objective': 'reg:squarederror',
                'eval_metric': 'rmse',
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'max_depth': trial.suggest_int('max_depth', 3, 10), # Có thể tăng max_depth
                'gamma': trial.suggest_float('gamma', 0.0, 1.0),
                'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 10.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
                'subsample': trial.suggest_float('subsample', 0.5, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            }

            horizon_results = {} # Lưu kết quả chi tiết của trial này
            overall_rmse_list = [] # Lưu RMSE trung bình của MỖI horizon
            global_step = 0 # Dùng cho Pruning

            # Lặp qua các horizon (ví dụ: h=1, h=3, h=6...)
            for h in self.horizons:
                scores = []
                fold_results = []

                # Lặp qua các fold (ví dụ: 5 folds)
                for i, ((X_train, y_train), (X_val, y_val)) in enumerate(self.walkfolds[h]):

                    model = xgb.XGBRegressor(
                        **params,
                        n_estimators=1000, # Có thể tăng cho dữ liệu hourly
                        random_state=42,
                        tree_method='hist', # Dùng 'gpu_hist' nếu có GPU
                        early_stopping_rounds=100 # Giữ nguyên
                    )

                    model.fit(
                        X_train,
                        y_train,
                        eval_set=[(X_val, y_val)],
                        verbose=False
                    )

                    preds = model.predict(X_val)
                    
                    # Tính toán các chỉ số
                    rmse = np.sqrt(mean_squared_error(y_val, preds))

                    ss_res = np.sum((y_val - preds) ** 2)
                    ss_tot = np.sum((y_val - np.mean(y_val)) ** 2)
                    r2 = 1 - ss_res / ss_tot if ss_tot != 0 else 0.0

                    scores.append(rmse)
                    fold_results.append({'rmse': rmse, 'r2': r2})

                    # Báo cáo cho Optuna (Pruning)
                    trial.report(rmse, step=global_step)
                    if trial.should_prune():
                        print(f"Trial {trial.number} pruned tại global_step={global_step}")
                        raise optuna.TrialPruned()
                    global_step += 1

                # Tính RMSE trung bình cho horizon H
                mean_rmse_h = float(np.mean(scores))
                horizon_results[h] = {
                    'mean_rmse': mean_rmse_h,
                    'fold_scores': fold_results,
                }
                overall_rmse_list.append(mean_rmse_h) # Thêm vào danh sách chung

            # Tính RMSE trung bình của các RMSE trung bình
            overall_mean_rmse = float(np.mean(overall_rmse_list))
            print(
                f"Trial {trial.number} finished, "
                f"overall mean RMSE across horizons={overall_mean_rmse:.4f}\n"
            )

            # Lưu lại lịch sử để debug
            self.fold_history.append({
                'trial': trial.number,
                'params': params,
                'horizon_results': horizon_results,
                'overall_mean_rmse': overall_mean_rmse,
            })
            
            # Trả về 1 con số duy nhất để Optuna tối ưu
            return overall_mean_rmse

        return objective

    # ======================== 4. RUN OPTUNA ========================

    def run_optuna(self, n_trials=50):
        """
        Chạy quá trình tối ưu hóa Optuna.
        """
        if not self.walkfolds[self.horizons[0]]:
            raise ValueError("Chưa tạo folds. Hãy gọi add_target_shifts() và create_walkforward_folds() trước.")
            
        objective_fn = self.create_objective()

        study = optuna.create_study(
            direction='minimize', # Tối ưu (giảm) RMSE
            pruner=optuna.pruners.MedianPruner(
                n_warmup_steps=len(self.horizons) # Chờ ít nhất 1 horizon hoàn thành
            )
        )
        study.optimize(objective_fn, n_trials=n_trials)
        self.best_params = study.best_params
        return study

    # ======================== 5. TRAIN A FINAL MODEL PER HORIZON ========================

    def train_final_models(self):
        """
        Huấn luyện mô hình cuối cùng cho TỪNG horizon
        sử dụng bộ tham số tốt nhất (best_params) tìm được từ Optuna
        trên TOÀN BỘ self.df.
        """
        if self.best_params is None:
            raise ValueError("Bạn phải chạy run_optuna() trước khi train_final_models().")

        final_params = self.best_params.copy()
        final_params.update({
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'n_estimators': 1500, # Có thể tăng n_estimators cho mô hình cuối cùng
            'early_stopping_rounds': 150 # Dùng (X_full, y_full_h) làm eval_set
        })

        X_full = self.df[self.X_cols]

        for h in self.horizons:
            print(f"Đang huấn luyện mô hình cuối cùng cho horizon h={h}...")
            y_full_h = self.df[f"{self.target_col}_h{h}"]

            model_h = xgb.XGBRegressor(
                **final_params,
                random_state=42,
                tree_method='hist' # Dùng 'gpu_hist' nếu có GPU
            )

            # Sử dụng early stopping ngay cả trên mô hình cuối cùng
            # bằng cách dùng chính tập full làm eval_set
            model_h.fit(
                X_full, 
                y_full_h, 
                eval_set=[(X_full, y_full_h)],
                verbose=False
            )
            
            self.final_models[h] = model_h
            print(f"Final model cho horizon {h} đã huấn luyện xong.")

    # ======================== 6. PREDICTION HELPERS ========================

    def predict_horizon(self, X_today, h):
        """
        Dự đoán cho MỘT horizon H, sử dụng MỘT dòng/nhiều dòng dữ liệu đầu vào.
        """
        if h not in self.horizons:
            raise ValueError(f"Horizon {h} không nằm trong {self.horizons}.")
        if self.final_models[h] is None:
            raise ValueError(f"Model cho horizon {h} chưa được huấn luyện. Gọi train_final_models().")

        # Đảm bảo X_today là DataFrame và chỉ lấy các cột cần thiết
        if isinstance(X_today, pd.Series):
             X_input = X_today[self.X_cols].to_frame().T
        else:
             X_input = X_today[self.X_cols]
             
        return self.final_models[h].predict(X_input)

    def predict_all_horizons(self, X_today):
        """
        Dự đoán cho TẤT CẢ các horizon.
        """
        results = {}
        for h in self.horizons:
            results[h] = self.predict_horizon(X_today, h)
        return results

    # ======================== 6b. BUILD PREDICTIONS FRAME (Plot) ========================

    def get_predictions_frame(self, df_with_shift, h):
        """
        Tạo một DataFrame chứa (y_true, y_pred, feature_time, target_time)
        cho một horizon H cụ thể, dựa trên một DataFrame đầu vào.
        """
        if h not in self.horizons:
            raise ValueError(f"Horizon {h} không nằm trong {self.horizons}.")
        if self.final_models[h] is None:
            raise ValueError(f"Model cho horizon {h} chưa được huấn luyện.")

        X = df_with_shift[self.X_cols]
        y_true = df_with_shift[f"{self.target_col}_h{h}"]
        y_pred = self.final_models[h].predict(X)

        feature_time = df_with_shift[self.date_col]
        
        # Đảm bảo cột target_time tồn tại
        if f"{self.date_col}_h{h}" in df_with_shift.columns:
            target_time = df_with_shift[f"{self.date_col}_h{h}"]
        else:
            # Nếu không có, tự tính toán (chậm hơn)
            target_time = pd.to_datetime(feature_time) + pd.to_timedelta(h, unit='h')

        df_pred = pd.DataFrame({
            "feature_time": feature_time.values,
            "target_time": target_time.values,
            "y_true": y_true.values,
            "y_pred": y_pred,
        })
        return df_pred

    def plot_predictions(self, df_with_shift, h, use_target_time=True, n_points=None):
        """
        Vẽ biểu đồ so sánh True vs Predicted cho horizon H.
        """
        df_pred = self.get_predictions_frame(df_with_shift, h)

        df_plot = df_pred.copy()
        if use_target_time and "target_time" in df_plot.columns:
            df_plot["x_axis"] = pd.to_datetime(df_plot["target_time"])
            xlabel = f"Target Time (Dự đoán cho lúc...)"
        else:
            df_plot["x_axis"] = pd.to_datetime(df_plot["feature_time"])
            xlabel = f"Feature Time (Dự đoán từ lúc...)"

        if n_points is not None:
            df_plot = df_plot.iloc[-n_points:] # Lấy N điểm cuối cùng

        plt.figure(figsize=(15, 5)) # Tăng kích thước
        plt.plot(df_plot["x_axis"], df_plot["y_true"], label="True", alpha=0.8)
        plt.plot(df_plot["x_axis"], df_plot["y_pred"], label="Predicted", linestyle='--')
        plt.xlabel(xlabel)
        plt.ylabel(self.target_col)
        plt.title(f"Dự báo cho tầm nhìn {h} giờ (Horizon h={h})")
        plt.legend()
        plt.grid(True, linestyle=':', alpha=0.6)
        plt.tight_layout()
        plt.show()

    # ======================== 7. EVAL ON TRAIN SET (CHECK OVERFIT) ========================

    def evaluate_train_models(self):
        """
        Đánh giá các mô hình cuối cùng trên TOÀN BỘ tập huấn luyện (self.df)
        để kiểm tra overfitting.
        """
        print("Đang đánh giá mô hình trên toàn bộ tập TRAIN (để kiểm tra overfit)...")
        if any(self.final_models[h] is None for h in self.horizons):
            raise ValueError("Tất cả models phải được huấn luyện trước khi đánh giá.")

        metrics = {}
        X_train_full = self.df[self.X_cols]

        for h in self.horizons:
            y_train_h = self.df[f"{self.target_col}_h{h}"]
            preds = self.final_models[h].predict(X_train_full)

            mse = mean_squared_error(y_train_h, preds)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_train_h, preds)
            eps = 1e-8
            mape = np.mean(np.abs((y_train_h - preds) / (y_train_h + eps))) * 100

            ss_res = np.sum((y_train_h - preds) ** 2)
            ss_tot = np.sum((y_train_h - np.mean(y_train_h)) ** 2)
            r2 = 1 - ss_res / ss_tot if ss_tot != 0 else 0.0

            print(
                f"[TRAIN] Horizon h={h} -> "
                f"RMSE: {rmse:.4f}, R²: {r2:.4f}, MAE: {mae:.4f}"
            )

            metrics[h] = {
                'rmse': rmse, 'mae': mae, 'mape': mape, 'mse': mse, 'r2': r2,
            }

        return metrics

    # ======================== 8. EVALUATION ON A TEST SET ========================

    def evaluate_final_models(self, test_df):
        """
        Đánh giá các mô hình cuối cùng trên một tập TEST (hold-out) bên ngoài.
        """
        print("Đang đánh giá mô hình trên tập TEST (hold-out)...")
        if any(self.final_models[h] is None for h in self.horizons):
            raise ValueError("Tất cả models phải được huấn luyện trước khi đánh giá.")

        metrics = {}
        X_test = test_df[self.X_cols]

        for h in self.horizons:
            y_test_h = test_df[f"{self.target_col}_h{h}"]
            preds = self.final_models[h].predict(X_test)

            mse = mean_squared_error(y_test_h, preds)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_test_h, preds)
            eps = 1e-8
            mape = np.mean(np.abs((y_test_h - preds) / (y_test_h + eps))) * 100

            ss_res = np.sum((y_test_h - preds) ** 2)
            ss_tot = np.sum((y_test_h - np.mean(y_test_h)) ** 2)
            r2 = 1 - ss_res / ss_tot if ss_tot != 0 else 0.0

            print(
                f"[TEST] Horizon h={h} -> "
                f"RMSE: {rmse:.4f}, R²: {r2:.4f}, MAE: {mae:.4f}"
            )

            metrics[h] = {
                'rmse': rmse, 'mae': mae, 'mape': mape, 'mse': mse, 'r2': r2,
            }

        return metrics

    # ======================== 9. SAVE FINAL MODELS (PICKLE) ========================

    def save_all_final_models_pkl(self, base_filename_prefix):
        """
        Lưu tất cả các model cuối cùng (final_models) ra đĩa dùng PICKLE (.pkl).
        """
        if any(self.final_models[h] is None for h in self.horizons):
            raise ValueError("Phải train model trước khi lưu. Hãy gọi train_final_models().")
        
        saved_paths = {}
        for h in self.horizons:
            model = self.final_models[h]
            # Tên file sẽ là prefix + _h1.pkl, _h3.pkl ...
            filename = f"{base_filename_prefix}_h{h}.pkl"
            
            try:
                with open(filename, 'wb') as f: 
                    pickle.dump(model, f)
                saved_paths[h] = filename
                print(f"Model (Pickle) cho horizon {h} đã được lưu vào: {filename}")
            except Exception as e:
                print(f"Lỗi khi lưu file pickle {filename}: {e}")
        
        return saved_paths

    # ======================== 10. SAVE RESULTS SUMMARY (JSON) ========================
    
    def save_results_to_json(
        self,
        filename,
        model_name,
        train_metrics,
        test_metrics,
        saved_model_paths=None
    ):
        """
        Lưu kết quả tóm tắt (best param, metrics, timestamp,...) vào 1 file JSON.
        """
        
        results_summary = {
            "model_name": model_name,
            "run_timestamp_utc": datetime.utcnow().isoformat(), # Đã sửa (xóa 1 .datetime)
            "target_column": self.target_col,
            "feature_columns": self.X_cols,
            "horizons_hours": self.horizons, # Đổi tên cho rõ
            "cross_validation_setup": {
                "n_splits": self.n_splits,
                "test_size_per_fold_hours": self.test_size, # Đổi tên
                "gap_size_hours": self.gap_size_hours, # THÊM
                "mode": self.mode,
            },
            "optuna_best_params": self.best_params,
            "evaluation_train_set": train_metrics,
            "evaluation_test_set": test_metrics,
            "saved_model_files": saved_model_paths if saved_model_paths else {},
            "optuna_fold_history": self.fold_history 
        }

        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(results_summary, f, indent=4, cls=NpEncoder)
            print(f"Kết quả tóm tắt đã được lưu vào: {filename}")
        except Exception as e:
            print(f"Lỗi khi lưu file JSON: {e}")

In [25]:
XGBoost_pipeline_hourly = MultiHorizonHourly_WalkForwardOptuna_XGBoost_Pipeline(
    df=train_dataset_h,
    date_col="datetime",
    target_col="temp",
    feature_cols=[col for col in train_dataset_h.columns if col not in ["datetime", "temp", "temp_next", "datetime_next"]],
    n_splits=5,
    test_size=365,
    mode="expanding",
    horizons=(1, 2, 3, 4, 5)
)

In [26]:
XGBoost_pipeline_hourly.add_target_shifts()
XGBoost_pipeline_hourly.create_walkforward_folds()

Đang tạo các cột target (shifted)...
Hình dạng DF sau khi shift và dropna: (69176, 130)
Đang tạo 5 folds, mỗi fold test_size=365h, gap=24h...
Đã tạo thành công 5 folds.


In [27]:
XGBoost_pipeline_hourly.run_optuna(n_trials=10)

[I 2025-11-17 03:56:45,095] A new study created in memory with name: no-name-66990aea-3443-474a-8317-dcf811da1cf2
[I 2025-11-17 03:59:47,677] Trial 0 finished with value: 1.0326366206486297 and parameters: {'learning_rate': 0.024591455082855894, 'max_depth': 5, 'gamma': 0.6841974344303405, 'reg_lambda': 9.850590045406502, 'reg_alpha': 0.6972319899671586, 'subsample': 0.5086579172327925, 'colsample_bytree': 0.7315779926989582, 'min_child_weight': 2}. Best is trial 0 with value: 1.0326366206486297.


Trial 0 finished, overall mean RMSE across horizons=1.0326



[I 2025-11-17 04:06:03,661] Trial 1 finished with value: 1.0378910830344381 and parameters: {'learning_rate': 0.010896748358799948, 'max_depth': 7, 'gamma': 0.9289916129638199, 'reg_lambda': 6.174018531061216, 'reg_alpha': 5.927658601625048, 'subsample': 0.7141180672699262, 'colsample_bytree': 0.7685013579789269, 'min_child_weight': 7}. Best is trial 0 with value: 1.0326366206486297.


Trial 1 finished, overall mean RMSE across horizons=1.0379



[I 2025-11-17 04:14:25,829] Trial 2 finished with value: 1.0519385172182194 and parameters: {'learning_rate': 0.03690869237740869, 'max_depth': 10, 'gamma': 0.253942191399806, 'reg_lambda': 6.994355272352963, 'reg_alpha': 7.970487392983752, 'subsample': 0.6587798769110595, 'colsample_bytree': 0.6837316439184529, 'min_child_weight': 1}. Best is trial 0 with value: 1.0326366206486297.


Trial 2 finished, overall mean RMSE across horizons=1.0519



[W 2025-11-17 04:23:36,559] Trial 3 failed with parameters: {'learning_rate': 0.014152240715301304, 'max_depth': 9, 'gamma': 0.5957862159075105, 'reg_lambda': 9.422952506105066, 'reg_alpha': 6.950303974168369, 'subsample': 0.9375311549150219, 'colsample_bytree': 0.5463644575658877, 'min_child_weight': 5} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\ASUS\anaconda3\envs\wp_env\Lib\site-packages\optuna\study\_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\ASUS\AppData\Local\Temp\ipykernel_30928\2100940939.py", line 273, in objective
    model.fit(
  File "c:\Users\ASUS\anaconda3\envs\wp_env\Lib\site-packages\xgboost\core.py", line 774, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "c:\Users\ASUS\anaconda3\envs\wp_env\Lib\site-packages\xgboost\sklearn.py", line 1365, in fit
    self._Booster = train(
                    ^^^^^^
  Fil

KeyboardInterrupt: 